這章節主要是為了帶領大家玩資料，會有一些實用的爬蟲，然後用案例無壓力地慢慢拆解唷

## 1.資料擷取

取得資料有幾種方式
 - 已經整理好的資料集，如政府資料開放平台[OpenData](https://data.gov.tw/)
 - 用該網頁的API（像是googlemap API）
 - 直接用爬蟲擷取網站
   - 擷取網站又有分靜態網站可以輕易地擷取，也有網站資料寫在AJAX不容易擷取的(超出本次課程範圍)，以下用常用的社群媒體及新聞網站資料做介紹。

### 1-1 PTT個版文章搜尋

台灣最大BBS交流平台PTT已經有網頁版本，而且規律的結構、熱門的討論很適合做案例分析。

In [1]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup

In [2]:
board ='NBA' #版名，也可以嘗試movie、Beauty、TaichungBun等板
url = 'https://www.ptt.cc/bbs/'+board+'/index.html'

response = requests.get(url)
html_doc = response.text # text 屬性就是 html 檔案
soup = BeautifulSoup(response.text, "lxml") # 指定 lxml 作為解析器

author_ids = []  # 建立一個空的 list 來放作者 id
recommends = []  # 建立一個空的 list 來放推文數
post_titles = [] # 建立一個空的 list 來放文章標題
post_dates = []  # 建立一個空的 list 來放發文日期

posts = soup.find_all("div", class_ = "r-ent")
for post in posts:
    try:
        author_ids.append(post.find("div", class_ = "author").string)    
    except:
        author_ids.append(np.nan)
    try:
        post_titles.append(post.find("a").string)
    except:
        post_titles.append(np.nan)
    try:
        post_dates.append(post.find("div", class_ = "date").string)
    except:
        post_dates.append(np.nan)

# 推文數藏在 div 裡面的 span 所以分開處理
recommendations = soup.find_all("div", class_ = "nrec")
for recommendation in recommendations:
    try:
        recommends.append(int(recommendation.find("span").string))
    except:
        recommends.append(np.nan)
        
ptt_dict = {"author": author_ids,
                "recommends": recommends,
                "title": post_titles,
                "date": post_dates
}

ptt_df = pd.DataFrame(ptt_dict)
ptt_df

,author,date,recommends,title
0,ericf129,4/18,24.0,[公告] NBA 板 開始舉辦樂透!
1,MrSatan,4/18,27.0,Re: [新聞] 詹姆斯第四度入選 時代百大唯一籃球員
2,pneumo,4/18,68.0,[花邊] James Harden晃飛Rubio後抖肩...但沒進
3,huntai,4/18,8.0,Re: [花邊] 狀元不選Zion？West：那就像錯過MJ一樣
4,LeBronJame23,4/18,3.0,Re: [花邊] 狀元不選Zion？West：那就像錯過MJ一樣
5,namie810303,4/07,80.0,[公告] 季後賽期間條款
6,Vedan,4/08,NaN,[公告] 板規v6.3
7,JerroLi,4/12,21.0,[情報] Playoff Schedule 18–19
8,ericf129,4/18,28.0,[公告] NBA 板 開始舉辦樂透!


### 1-2 PTT個版搜尋-方法2

In [3]:
import requests
import time
import json
from bs4 import BeautifulSoup


PTT_URL = 'https://www.ptt.cc'

def get_web_page(url):
    resp = requests.get(
        url=url,
        cookies={'over18': '1'}  # 告知Server已回答過滿18歲的問題
    )
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text


def get_articles(dom, date):
    soup = BeautifulSoup(dom, 'html5lib')

    # 取得上一頁的連結
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']

    articles = []  # 儲存取得的文章資料
    divs = soup.find_all('div', 'r-ent')
    for d in divs:
        if d.find('div', 'date').text.strip() == date:  # 發文日期正確
            # 取得推文數
            push_count = 0
            push_str = d.find('div', 'nrec').text
            if push_str:
                try:
                    push_count = int(push_str)  # 轉換字串為數字
                except ValueError:
                    # 若轉換失敗，可能是'爆'或 'X1', 'X2', ...
                    # 若不是, 不做任何事，push_count 保持為 0
                    if push_str == '爆':
                        push_count = 99
                    elif push_str.startswith('X'):
                        push_count = -10

            # 取得文章連結及標題
            if d.find('a'):  # 有超連結，表示文章存在，未被刪除
                href = d.find('a')['href']
                title = d.find('a').text
                author = d.find('div', 'author').text if d.find('div', 'author') else ''
                articles.append({
                    'title': title,
                    'href': href,
                    'push_count': push_count,
                    'author': author
                })
    return articles, prev_url


def get_author_ids(posts, pattern):
    ids = set()
    for post in posts:
        if pattern in post['author']:
            ids.add(post['author'])
    return ids


if __name__ == '__main__':
    board ='Gossiping' #版名，也可以嘗試movie、Beauty、TaichungBun等板
    current_page = get_web_page(PTT_URL + '/bbs/'+board+'/index.html')
    if current_page:
        articles = []  # 全部的今日文章
        today = time.strftime("%m/%d").lstrip('0')  # 今天日期, 去掉開頭的 '0' 以符合 PTT 網站格式
        current_articles, prev_url = get_articles(current_page, today)  # 目前頁面的今日文章
        while current_articles:  # 若目前頁面有今日文章則加入 articles，並回到上一頁繼續尋找是否有今日文章
            articles += current_articles
            current_page = get_web_page(PTT_URL + prev_url)
            current_articles, prev_url = get_articles(current_page, today)

        # 印出所有不同的 5566 id
#         print(get_author_ids(articles, '5566'))

        # 儲存或處理文章資訊
        print('今天有', len(articles), '篇文章')
        threshold = 98
        print('熱門文章(> %d 推):' % (threshold))
        for a in articles:
            if int(a['push_count']) > threshold:
                print(a)
        with open('./data/gossiping.json', 'w', encoding='utf-8') as f:
            json.dump(articles, f, indent=2, sort_keys=True, ensure_ascii=False)

今天有 401 篇文章
熱門文章(> 98 推):
{'title': '[新聞] 外媒看衰？那些年 郭台銘從沒兌現的承諾', 'href': '/bbs/Gossiping/M.1555570420.A.BAF.html', 'push_count': 99, 'author': 'Gocoba'}
{'title': '[新聞] 氣象局：非正常能量釋放 周內恐有規', 'href': '/bbs/Gossiping/M.1555568984.A.E22.html', 'push_count': 99, 'author': 'justice2008'}
{'title': '[問卦] 《與惡》是不是完全貼切描述台灣媒體現況', 'href': '/bbs/Gossiping/M.1555567330.A.845.html', 'push_count': 99, 'author': 'Bjergsen'}
{'title': '[臉書] 蔡英文', 'href': '/bbs/Gossiping/M.1555567874.A.DAB.html', 'push_count': 99, 'author': 'ptt12'}
{'title': '[爆卦] 宿舍484要倒了?', 'href': '/bbs/Gossiping/M.1555565732.A.80B.html', 'push_count': 99, 'author': 'stan871125'}
{'title': 'Re: [爆卦] 馬路裂開了', 'href': '/bbs/Gossiping/M.1555565597.A.E3E.html', 'push_count': 99, 'author': 'killeryuan'}
{'title': '[爆卦] 馬路裂開了（誤傳）柏油灑落（O)', 'href': '/bbs/Gossiping/M.1555565036.A.CCF.html', 'push_count': 99, 'author': 'Abet'}
{'title': '[問卦] 這次地震有多少人真的有恐懼感的', 'href': '/bbs/Gossiping/M.1555564529.A.675.html', 'push_count': 99, 'author': 'misaka10

In [4]:
ptt_df = pd.DataFrame(articles)
filter = ptt_df["push_count"] > 98
ptt_df[filter]

,author,href,push_count,title
83,Gocoba,/bbs/Gossiping/M.1555570420.A.BAF.html,99,[新聞] 外媒看衰？那些年 郭台銘從沒兌現的承諾
133,justice2008,/bbs/Gossiping/M.1555568984.A.E22.html,99,[新聞] 氣象局：非正常能量釋放 周內恐有規
179,Bjergsen,/bbs/Gossiping/M.1555567330.A.845.html,99,[問卦] 《與惡》是不是完全貼切描述台灣媒體現況
195,ptt12,/bbs/Gossiping/M.1555567874.A.DAB.html,99,[臉書] 蔡英文
240,stan871125,/bbs/Gossiping/M.1555565732.A.80B.html,99,[爆卦] 宿舍484要倒了?
271,killeryuan,/bbs/Gossiping/M.1555565597.A.E3E.html,99,Re: [爆卦] 馬路裂開了
276,Abet,/bbs/Gossiping/M.1555565036.A.CCF.html,99,[爆卦] 馬路裂開了（誤傳）柏油灑落（O)
340,misaka10032,/bbs/Gossiping/M.1555564529.A.675.html,99,[問卦] 這次地震有多少人真的有恐懼感的
354,syxuan,/bbs/Gossiping/M.1555564313.A.F3F.html,99,[問卦] 簡訊先來的舉手
395,ice0110,/bbs/Gossiping/M.1555563861.A.E00.html,99,[問卦] 簡訊通知呢?


In [5]:
def ptt_hot():
    target_url = 'http://disp.cc/b/PttHot'
    print('Start parsing pttHot....')
    rs = requests.session()
    res = rs.get(target_url, verify=False)
    soup = BeautifulSoup(res.text, 'html.parser')
    content = ""
    for data in soup.select('#list div.row2 div span.listTitle'):
        title = data.text
        link = "http://disp.cc/b/" + data.find('a')['href']
        if data.find('a')['href'] == "796-59l9":
            break
        content += '{} {}\n'.format(title, link)
    return content

pd.DataFrame(ptt_hot().split("\n")).head()

Start parsing pttHot....


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,0
0,■ [新聞] 文化瑰寶付之一炬！《刺客教條》遊戲內容將有助重建 http://disp.cc...
1,■ [問卦] 標太郎水餃告訴大眾 切勿相信網路謠言 http://disp.cc/b/796...
2,■ [爆卦] 政經造反啦！！！ http://disp.cc/b/796-b35p
3,■ [閒聊] 閃耀暖暖-長得醜錯了嗎QQ http://disp.cc/b/796-bdE7
4,■ Re: [問卦] 蜂蜜快到期了 該怎辦？(附圖) http://disp.cc/b/79...


### 1-4 Dcard熱門文章搜尋

In [6]:
import re
import requests
from bs4 import BeautifulSoup

def dcard_f():
    URL = 'https://www.dcard.tw/f'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    resp = requests.get(URL, headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')

    content = ""
    # 利用 regex 找出所有 'PostList_wrapper_' 開頭的 div
    for index, div in enumerate(soup.find_all('div', re.compile('PostEntry_content_\w{5}'))):
        if index == 10:
            return content
        title = div.h3.text.strip()
        excerpt = div.find_all('div')[0].text.strip()
        href = div.parent.parent['href']
        content += ' https://www.dcard.tw{}\n'.format(href)
    return content

dcard_f().split("\n")

[' https://www.dcard.tw/f/disaster/p/231113837-各地災情回報-[4／18-13:01-地震快報]',
 ' https://www.dcard.tw/f/funny/p/231106081-被媽媽衝康，差點被警察抓去勒戒。',
 ' https://www.dcard.tw/f/trending/p/231106354-普悠瑪駕駛被撤銷駕照終身禁考',
 ' https://www.dcard.tw/f/food/p/231106558-跟男友同居一年，兩人胖十五公斤！-＃自己煮',
 ' https://www.dcard.tw/f/girl/p/231105751-＃求解-換包包還是換男友？',
 ' https://www.dcard.tw/f/entertainer/p/231107410-更）超親民的議員',
 ' https://www.dcard.tw/f/mood/p/231104628-幹破林娘胰臟癌',
 ' https://www.dcard.tw/f/entertainer/p/231109455-辜莞允散播閨蜜裸照',
 ' https://www.dcard.tw/f/vehicle/p/231104856-還是有人不願意乖乖兩段式左轉',
 ' https://www.dcard.tw/f/relationship/p/231098894-你身邊也有難搞的公主們嗎？「等等吃什麼」的世紀難題，解法底加！ft.foodpanda',
 '']

In [7]:
dcard_df = pd.DataFrame(dcard_f().split("\n"))
dcard_df

,0
0,https://www.dcard.tw/f/disaster/p/231113837-各...
1,https://www.dcard.tw/f/funny/p/231106081-被媽媽衝...
2,https://www.dcard.tw/f/trending/p/231106354-普...
3,https://www.dcard.tw/f/food/p/231106558-跟男友同居...
4,https://www.dcard.tw/f/girl/p/231105751-＃求解-換...
5,https://www.dcard.tw/f/entertainer/p/23110741...
6,https://www.dcard.tw/f/mood/p/231104628-幹破林娘胰臟癌
7,https://www.dcard.tw/f/entertainer/p/23110945...
8,https://www.dcard.tw/f/vehicle/p/231104856-還是...
9,https://www.dcard.tw/f/relationship/p/2310988...


### 1-5 蘋果日報即時新聞

In [8]:
def apple_news():
    target_url = 'https://tw.appledaily.com/new/realtime'
    print('Start parsing appleNews....')
    rs = requests.session()
    res = rs.get(target_url, verify=False)
    soup = BeautifulSoup(res.text, 'html.parser')
    content = ""
    for index, data in enumerate(soup.select('.rtddt a'), 0):
        if index == 5:
            return content
        link = data['href']
        content += ' {}\n'.format(link)
    return content

apple_news().split("\n")

Start parsing appleNews....


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[' https://tw.news.appledaily.com/international/realtime/20190418/1552470/',
 ' https://tw.news.appledaily.com/international/realtime/20190418/1552676/',
 ' https://tw.lifestyle.appledaily.com/gadget/realtime/20190418/1552401/',
 ' https://tw.finance.appledaily.com/realtime/20190418/1552750/',
 ' https://tw.news.appledaily.com/politics/realtime/20190418/1552749/',
 '']

In [9]:
def movie():
    target_url = 'http://www.atmovies.com.tw/movie/next/0/'
    print('Start parsing movie ...')
    rs = requests.session()
    res = rs.get(target_url, verify=False)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')
    content = ""
    for index, data in enumerate(soup.select('ul.filmNextListAll a')):
        if index == 20:
            return content
        title = data.text.replace('\t', '').replace('\r', '')
        link = "http://www.atmovies.com.tw" + data['href']
        content += '{} {} '.format(title, link)
    return content

movie().split("\n")

Start parsing movie ...


['',
 '刀說異數：第14章 http://www.atmovies.com.tw/movie/fwcn77479124/ ',
 '牛津解密 http://www.atmovies.com.tw/movie/fpen55932728/ ',
 '原本以為只是手機掉了 http://www.atmovies.com.tw/movie/fsjp98531044/ ',
 '希特勒對畢卡索：被奪取的名畫 http://www.atmovies.com.tw/movie/fhit98016764/ ',
 '淪落人 http://www.atmovies.com.tw/movie/fscn38497794/ ',
 '上流世界 http://www.atmovies.com.tw/movie/flit26748466/ ',
 '失控班！ http://www.atmovies.com.tw/movie/fsfr37175992/ ',
 '就是要妳愛上我 http://www.atmovies.com.tw/movie/fljp88718066/ ',
 '電影刀劍亂舞 http://www.atmovies.com.tw/movie/fejp68493238/ ',
 '魔力女聲 http://www.atmovies.com.tw/movie/ften46483364/ ',
 '愛上太空女神 http://www.atmovies.com.tw/movie/faen65215088/ ',
 '噬魂實驗 http://www.atmovies.com.tw/movie/fden44685940/ ',
 'Aqours ASIA TOUR 2019 in Seoul http://www.atmovies.com.tw/movie/fljp93558479/ ',
 '刀說異數：第15章 http://www.atmovies.com.tw/movie/fwcn77479125/ ',
 '復仇者聯盟4：終局之戰 http://www.atmovies.com.tw/movie/faen64154796/ ',
 '我唾棄你的墳墓3 http://www.atmovies.com.tw/movie/fien84530884/ ',
 '卡拉瓦喬：靈魂與血肉之軀

### 1-6 油價查詢

In [10]:
def oil_price():
    target_url = 'https://gas.goodlife.tw/'
    rs = requests.session()
    res = rs.get(target_url, verify=False)
    soup = BeautifulSoup(res.text, 'html.parser')
    title = soup.select('#main')[0].text.replace('\n', '').split('(')[0].replace('\n', ',')
    gas_price = soup.select('#gas-price')[0].text.replace('\n\n\n', '').replace(' ', '')
    cpc = soup.select('#cpc')[0].text.replace(' ', '')
    content = '{} {} {}'.format(title, gas_price, cpc)


    return content

oil_price().replace('\n', '')

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


'最後更新時間: 2019-04-18 15:14  柴油預計調整:-0.0元中油累計吸0.1元,若不列入計算,應調整:+0.8元受到鄰國油價影響，下週一2019年04月22日起,預計汽油每公升:漲0.5元 今日中油油價92:27.995油價:29.498:31.4柴油:26.7'

In [11]:
def technews():
    target_url = 'https://technews.tw/'
    print('Start parsing movie ...')
    rs = requests.session()
    res = rs.get(target_url, verify=False)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')
    content = ""

    for index, data in enumerate(soup.select('article div h1.entry-title a')):
        if index == 5:
            return content
        title = data.text
        link = data['href']
        content += '{} {}\n'.format(title, link)
    return content

technews().split("\n")

Start parsing movie ...


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


['大麻漢堡來了，4/20 大麻日美速食店限日嘗鮮 https://technews.tw/2019/04/18/rocky-mountain-high-cheeseburger-delight/',
 '英特爾新顯卡或用三星記憶體？高層訪南韓，揣測滿天飛 https://technews.tw/2019/04/18/intel-gpu-samsung-memory/',
 '台積電 2019 年第 1 季 EPS 2.37 元符合預期，地震沒有造成任何災情 https://finance.technews.tw/2019/04/18/tsmc-2019q1-financial-meeting/',
 'Facebook 呼籲不要用最終點擊評估行銷成效，而是總業績增幅 https://technews.tw/2019/04/18/facebook-claim-retails-should-not-only-judge-by-clicks-but-by-total-performance-increase/',
 '花蓮強震，台電：核一二廠正常運轉 https://technews.tw/2019/04/18/taipower-nuclear-power-plant-safe/',
 '']

### reference

  - [line-bot-tutorial](https://github.com/willismax/line-bot-tutorial)
  - [Python 網路爬蟲與資料分析入門實戰 ](https://www.tenlong.com.tw/products/9789864343386)